In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [15]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch_prep_policy.gdb", 'policy_override.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])


In [16]:
# read in data
polygons = r'.\2019\policy_override_2019.gdb\override_polygons_2019'
polygons_df = pd.DataFrame.spatial.from_featureclass(polygons)
polygons_df.head()

,OBJECTID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,1,10.0,25.0,0.8,1.0,2037.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,Farmington Downtown,Neighborhood Center,2041,2050,1,,,,,"{""rings"": [[[425020.9299999997, 4536963.539999..."
1,2,2,30.0,100.0,2.0,4.0,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Station Park/North Station,Urban Center,2019,2030,1,,,,,"{""rings"": [[[422130.9299999997, 4539255.970000..."
2,3,3,20.0,25.0,0.8,1.0,2037.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Centerville Main Street,Neighborhood Center,2041,2050,1,,,,,"{""rings"": [[[426023.38999999966, 4529335.5], [..."
3,4,4,25.0,50.0,1.0,1.5,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,North Salt Lake Town Center,City Center,2031,2040,1,,,,,"{""curveRings"": [[[423047.6200000001, 4520431.4..."
4,5,5,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,00000000,None,WFRC,University of Utah,Special District,2019,2030,1,,,,,"{""rings"": [[[429590.76999999955, 4514131.05000..."


In [17]:
phased = polygons_df[polygons_df['phase_count'] > 1].copy()
phased

,OBJECTID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
29,30,30,65.0,NaN,0.0,NaN,NaN,1,1,0,1,1,0,0,0,11011000,CBDStaging,WFRC,Salt Lake City CBD,Metropolitan Center,2019,2030,3,"2019,2025,2035","65,95,125","0,0,0","11011000,11011000,11011000","{""rings"": [[[425525.2999999998, 4513678], [425..."


In [18]:
df = pd.DataFrame()
for index, row in phased.iterrows():
    phase_years = row['phase_years'].split(',')
    phase_max_duas = row['phase_maxduas'].split(',')
    phase_max_fars = row['phase_maxfars'].split(',')
    phase_types = row['phase_types'].split(',')

    # check to make sure phase lists are equal
    if len(set([len(phase_years), len(phase_max_duas), len(phase_max_fars), len(phase_types)])) != 1:
        raise ValueError("Missmatch in phase length values for id: {}".format(row['id']))

    
    for i in range(len(phase_years)):
        new_row = row
        new_row['year'] =  phase_years[i]
        new_row['max_dua'] =  phase_max_duas[i]
        new_row['max_far'] =  phase_max_fars[i]
        new_row['type1'] =  phase_types[i][0]
        new_row['type2'] =  phase_types[i][1]
        new_row['type3'] =  phase_types[i][2]
        new_row['type4'] =  phase_types[i][3]
        new_row['type5'] =  phase_types[i][4]
        new_row['type6'] =  phase_types[i][5]
        new_row['type7'] =  phase_types[i][6]
        new_row['type8'] =  phase_types[i][7]
        df = df.append(new_row)
        

In [19]:
non_phased = polygons_df[polygons_df['phase_count'] == 1].copy()
policy_sdf = non_phased.append(df)
policy_sdf.spatial.to_featureclass(location=os.path.join(gdb2, 'policy_override_polygons_2019_phased'),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Policy-Layer\\Outputs\\policy_override.gdb\\policy_override_polygons_2019_phased'

In [25]:
parcels = r'..\Parcels\2020-WFRC\Outputs\remm_base_year_2019.gdb\parcels_2019'
parcels_pts = arcpy.FeatureToPoint_management(parcels, os.path.join(gdb, '_02_parcels_pts'), "INSIDE")

parcels_pts_lyr = arcpy.MakeFeatureLayer_management(parcels_pts, 'parcels_pts_lyr') 
arcpy.SelectLayerByAttribute_management(parcels_pts_lyr, 'NEW_SELECTION', "building_type_id in (6,9,10,13,14,16)")

arcpy.DeleteFeatures_management(parcels_pts_lyr)

arcpy.DeleteField_management(parcels_pts_lyr, 
                             ["CO_NAME", "TAZID", "COUNTY_ID","PARCEL_ID", "TOTAL_MKT_VALUE", "land_value", "residential_units", "building_sqft","FLOORS_CNT", "year_built",
                              "EFFBUILT_YR", "IS_OUG","max_dua","max_far","max_height", "max_height", "type1", "type2", "type3", "type4","type5", "type6", "type7", "type8",
                              "agriculture", "basebldg","NoBuild","redev_friction", "building_type_id","x", "y", "parcel_acres","Split","Split_Factor","parcel_id_REMM_old",
                              "residential_price","non_residential_price","non_residential_sqft"])

<Result 'parcels_pts_lyr'>

In [21]:
# parcels_pts_df = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_02_parcels_pts'))
# parcels_pts_df = parcels_pts_df[['parcel_id_REMM','SHAPE']].copy()
# parcels_pts_df.spatial.to_featureclass(location=os.path.join(gdb, '_03_parcels_pts'),sanitize_columns=False)

In [26]:
# use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
target_features = parcels_pts_lyr
join_features = os.path.join(gdb2, 'policy_override_polygons_2019_phased')
output_features = os.path.join(gdb, "_03_parcels_pts_policy_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# max_dua
fieldindex = fieldmappings.findFieldMapIndex('max_dua')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)


# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_MANY', "KEEP_COMMON", 
                           fieldmappings, "INTERSECT")

In [28]:
parcel_pts_policy_sdf = pd.DataFrame.spatial.from_featureclass(sj[0])
parcel_pts_policy_sdf.head()

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,parcel_id_REMM,ORIG_FID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,1,1,113,1,1,114.0,25.0,50.0,1.3,1.5,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Bountiful Downtown,City Center,2019.0,2030.0,1.0,,,,,"{""x"": 425848.5560999997, ""y"": 4526928.1381, ""s..."
1,2,1,3,39,3,3,40.0,0.4,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,North Salt Lake Industrial District,Industrial District,2019.0,2030.0,1.0,,,,,"{""x"": 421713.90490000043, ""y"": 4524888.319, ""s..."
2,3,1,6,109,6,6,110.0,0.0,NaN,0.5,NaN,2019.0,0,0,0,1,1,0,0,0,00011000,WFRCCenters,WFRC,Sheep Lane Employment District,Employment District,2031.0,2040.0,1.0,,,,,"{""x"": 424126.7297, ""y"": 4534739.7322, ""spatial..."
3,4,1,13,39,13,13,40.0,0.4,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,North Salt Lake Industrial District,Industrial District,2019.0,2030.0,1.0,,,,,"{""x"": 421729.23759999964, ""y"": 4522131.6081000..."
4,5,1,16,39,16,16,40.0,0.4,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,North Salt Lake Industrial District,Industrial District,2019.0,2030.0,1.0,,,,,"{""x"": 421867.2581000002, ""y"": 4522303.5858, ""s..."


In [31]:
parcel_pts_policy_sdf = parcel_pts_policy_sdf[['parcel_id_REMM', 'max_dua','max_far','year','type1','type2','type3','type4','type5','type6','type7','type8','locnote','mponote']].copy()
df.rename(columns={'parcel_id_REMM': 'parcel_id'},axis=1, inplace=True)
parcel_pts_policy_sdf.to_csv(r'Outputs\zoning_parcels_p.csv', index=False)

In [24]:
# import subprocess
os.startfile(r".\Policy_Override_Review.aprx")